# Информационный поиск

Скачиваем классический набор данных -- набор текстов об аэронавтике CRANFIELD

In [1]:
! wget -q http://ir.dcs.gla.ac.uk/resources/test_collections/cran/cran.tar.gz
! tar -xvf cran.tar.gz
! rm cran.tar.gz*

cran.all.1400
cran.qry
cranqrel
cranqrel.readme


Берём только сами запросы (это будут наши документы)

In [2]:
! grep -v "^\." cran.qry > just.qry
! head -3 just.qry

what similarity laws must be obeyed when constructing aeroelastic models
of heated high speed aircraft .
what are the structural and aeroelastic problems associated with flight


Объединяем многострочные в один

In [3]:
raw_query_data = [line.strip() for line in open("just.qry", "r").readlines()]
raw_query_data[:5]

['what similarity laws must be obeyed when constructing aeroelastic models',
 'of heated high speed aircraft .',
 'what are the structural and aeroelastic problems associated with flight',
 'of high speed aircraft .',
 'what problems of heat conduction in composite slabs have been solved so']

In [4]:
query_data = [""]

for query_part in raw_query_data:
  query_data[-1] += query_part + " "
  if query_part.endswith("."):
    query_data.append("")

query_data[:2]

['what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft . ',
 'what are the structural and aeroelastic problems associated with flight of high speed aircraft . ']

### Составим запросы к нашим документам

In [58]:
#QUERIES = ['theory of bending', 'aeroelastic effects']
QUERIES = ['photoelastic materials', 'swept cylinders']

## Boolean retrieval
Представим каждый документ как "битовую маску": вектор размером со словарь, в котором на каждой позиции единица, если в документе есть соответсвующий терм, и ноль, если терма нет

In [33]:
# в разных версиях ответы могут отличаться, поэтому важно иметь одну и ту же
! pip install -q scikit-learn==0.22.2.post1

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for scikit-learn
ERROR: Could not build wheels for scikit-learn, which is required to install pyproject.toml-based projects


In [59]:
from  sklearn.feature_extraction.text import CountVectorizer

encoder = CountVectorizer(binary=True)
encoded_data = encoder.fit_transform(query_data)
encoded_queries = encoder.transform(QUERIES)
list(encoder.vocabulary_)[:5]

['what', 'similarity', 'laws', 'must', 'be']

In [37]:
encoded_data.shape

(227, 946)

Посмотрим на представление первого предложения

In [60]:
id2term = {idx: term for term, idx in encoder.vocabulary_.items()}
non_zero_values_ids = encoded_data[0].nonzero()[1]

terms = [id2term[idx] for idx in non_zero_values_ids]
terms

['what',
 'similarity',
 'laws',
 'must',
 'be',
 'obeyed',
 'when',
 'constructing',
 'aeroelastic',
 'models',
 'of',
 'heated',
 'high',
 'speed',
 'aircraft']


## Задание 0

Теперь для каждого из данных запросов `QUERIES` найдём ближайший для него документ из `query_data` по сходству Жаккара. Есть более эффективные способы это сделать, но вам требуется реализовать расстояние Жаккара и далее применить его к нашим данным.

In [61]:
import numpy as np

def jaccard_sim(vector_a: np.array, vector_b: np.array) -> float:
  """
    Сходство или коэффициент Жаккара: отношение мощности пересечения
    к мощности объединения
  """
  union = 0
  intersection = 0
  for i, x in enumerate(vector_a):
    if vector_b[i] == x and x == 1:
      intersection += 1
    if (x == 1) or (vector_b[i] == 1):
      union += 1
  return intersection / union

  return

#Проверка, что функция работает правильно
jaccard_sim(np.array([1, 0, 1, 0, 1]), np.array([0, 1, 1, 1, 1]))

0.4

Здесь документы представлены так же, как строки в матрице термов-документов. Каждая ячейка вектора отвечает за наличие/отсутствие конкретного элемента (например, слова-терма, когда у нас в словаре всего 5 слов). В первом случае их три, во втором — четыре. Объединение — все пять возможных элементов. Пересечение — два. Отсюда и 0.4.

## Задание 1
Теперь с помощью кода ниже вычислите для каждого запроса самые близкие документы.

In [62]:
for q_id, query in enumerate(encoded_queries):

  query = query.todense().A1
  docs = [doc.todense().A1 for doc in encoded_data]

  id2doc2similarity = [(doc_id, doc, jaccard_sim(query, doc)) for doc_id, doc in enumerate(docs)]

  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=True)

  print(f"Q: {QUERIES[q_id]}\nFOUND:")

  for closest_id, _, sim in closest[:3]:
    print(f"    {closest_id}\t{sim}\t{query_data[closest_id]}")


Q: photoelastic materials
FOUND:
    14	0.4	material properties of photoelastic materials . 
    156	0.08333333333333333	what qualitative and quantitative material is available on ablation materials research . 
    0	0.0	what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft . 
Q: swept cylinders
FOUND:
    62	0.2	where can i find pressure data on surfaces of swept cylinders . 
    148	0.09090909090909091	papers on small deflection theory for buckling of sandwich cylinders . 
    58	0.07692307692307693	how much is known about boundary layer flows along non-circular cylinders . 


Видим, что кое-где просачиваются  тексты, которых с запросами объединяют малозначительные термы, но при этом коэффициент Жаккара -- наша функция ранжирования! -- высок.

# VSM

Попробуем теперь сделать то же, но с tf-idf и косинусным расстоянием. Мы сделаем всё опять "руками", но "в реальной жизни" лучше использоватьесть эффективные реализации cosine distance, например, из библиотеки scipy.

In [63]:
from  sklearn.feature_extraction.text import TfidfVectorizer

# Совет: обязательно разберитесь с тем, какие возможности
# предоставляет tf-idf vectorizer, какие параметры за что отвечают

tfidf_encoder = TfidfVectorizer()
tfidf_encoded_data = tfidf_encoder.fit_transform(query_data)
tfidf_encoded_queries = tfidf_encoder.transform(QUERIES)

list(tfidf_encoder.vocabulary_)[:3]

['what', 'similarity', 'laws']

## Задание 2
Реализовать косинусное расстояние

In [64]:
import numpy as np

def cosine_distance(vector_a: np.array, vector_b: np.array) -> float:
  """
    Косинусное расстояние: единица минус отношение скалярного произведения
    на произведение L2-норм
  """
  dot_product = np.dot(vector_a, vector_b)

  norm_a = np.linalg.norm(vector_a)
  norm_b = np.linalg.norm(vector_b)

  if norm_a == 0 or norm_b == 0:
      return 1.0

  cosine_similarity = dot_product / (norm_a * norm_b)
  return 1 - cosine_similarity

#Проверка, что функция работает правильно
cosine_distance(np.array([1, 0, 1, 1, 1]), np.array([0, 0, 1, 0, 0])) # 0.5

0.5


Теперь вычислим ближайшие по косинусному расстоянию между векторными представлениями документов и запросов

In [65]:
for q_id, query in enumerate(tfidf_encoded_queries):

  query = query.todense().A1
  docs = [doc.todense().A1 for doc in tfidf_encoded_data]
  id2doc2similarity = [(doc_id, doc, cosine_distance(query, doc)) \
                       for doc_id, doc in enumerate(docs)]

  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=False)

  print(f"Q: {QUERIES[q_id]}\nFOUND:")

  for closest_id, _, sim in closest[:6]:
    print(f"    {closest_id}\t{sim}\t{query_data[closest_id]}")

Q: photoelastic materials
FOUND:
    14	0.26107661469385257	material properties of photoelastic materials . 
    156	0.7475693331390902	what qualitative and quantitative material is available on ablation materials research . 
    0	1.0	what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft . 
    1	1.0	what are the structural and aeroelastic problems associated with flight of high speed aircraft . 
    2	1.0	what problems of heat conduction in composite slabs have been solved so far . 
    3	1.0	can a criterion be developed to show empirically the validity of flow solutions for chemically reacting gas mixtures based on the simplifying assumption of instantaneous local chemical equilibrium . 
Q: swept cylinders
FOUND:
    62	0.4914073240758777	where can i find pressure data on surfaces of swept cylinders . 
    81	0.7900748449150106	how do kuchemann's and multhopp's methods for calculating lift distributions on swept wings in subsonic flow